# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [2]:
import matplotlib
# Specify renderer
# matplotlib.use('Agg')

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Boiler-plate settings for producing pub-quality figures
# 1 point = 1/72 inch
from cycler import cycler
matplotlib.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
matplotlib.rcParams.update({
    'figure.figsize': (8, 5)  # inches
    ,
    'font.size':
    22  # points
    ,
    'legend.fontsize':
    16  # points
    ,
    'lines.linewidth':
    1.5  # points
    ,
    'axes.linewidth':
    1.5  # points
    ,
    'text.usetex':
    True  # Use LaTeX to layout text
    ,
    'font.family':
    "serif"  # Use serifed fonts
    ,
    'xtick.major.size':
    10  # length, points
    ,
    'xtick.major.width':
    1.5  # points
    ,
    'xtick.minor.size':
    6  # length, points
    ,
    'xtick.minor.width':
    1  # points
    ,
    'ytick.major.size':
    10  # length, points
    ,
    'ytick.major.width':
    1.5  # points
    ,
    'ytick.minor.size':
    6  # length, points
    ,
    "xtick.minor.visible":
    True,
    "ytick.minor.visible":
    True,
    'font.weight':
    'bold',
    'ytick.minor.width':
    1  # points
    ,
    'font.serif': ("Times", "Palatino", "Computer Modern Roman",
                   "New Century Schoolbook", "Bookman"),
    'font.sans-serif':
    ("Helvetica", "Avant Garde", "Computer Modern Sans serif"),
    'font.monospace': ("Courier", "Computer Modern Typewriter"),
    'font.cursive':
    "Zapf Chancery"
})

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

In [5]:
import sys
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp

# ML
import sklearn

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

#Set the display format to be scientific for ease of analysis
# pd.options.display.float_format = '{:,.2g}'.format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# NLP
import gensim 
from gensim import models
from gensim.models import Word2Vec
import tensorflow 

import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [7]:
try:
    import subprocess
    gitd = os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir))
    githash = subprocess.check_output('git --git-dir={0:s} --work-tree={1:s} '\
              'rev-parse HEAD'.format(gitd+'/.git',gitd),shell=True).rstrip()
except:
    githash = 'No repo found'

print(githash)

No repo found


In [8]:
%load_ext watermark
%watermark -v -h -n -g -m -p jupyerlab,numpy,scipy,sklearn,pandas,matplotlib

Thu Jun 04 2020 

CPython 3.7.6
IPython 7.13.0

jupyerlab not installed
numpy 1.18.1
scipy 1.4.1
sklearn 0.22.1
pandas 1.0.3
matplotlib 3.1.3

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
host name  : C02X61QTJHD5
Git hash   : a945881b3f51b23983451f0cd2ade69e739a8412


In [9]:
import glob
# NYC_review_files = glob.glob('scraped_data_nodupl_biz/' + '1????_pet_*' + '_2020*.csv')
# print(len(NYC_review_files))
SF_review_files = glob.glob('scraped_data_nodupl_biz/' + '9????*csv')
print(len(SF_review_files))

34


In [20]:
# df.columns = [
#    'Ind', 'review_name', 'review_rating', 'review_date', 'review_text',
#    'biz_url', 'biz_rating', 'biz_name', 'biz_phone', 'mon', 'tues', 'wed',
#    'thurs', 'fri', 'sat', 'sun'
#]

def read_all_reviews_in_area(list_of_files):
    for iii, fff in enumerate(list_of_files):
        df = pd.read_csv(fff,
                         usecols=['review_name', 'review_rating', 'review_date', 'review_text',
        'biz_url', 'biz_rating', 'biz_name']
                        )  # nrows=5, chunksize=1000000,
                                    # dtype={"country":"category", "beer_servings":"float64"},
                                    # na_values=0.0
                                    # index_col=0, skiprows=1      # header
        if iii ==0: 
            df_all = df
        else:
            df_all= pd.concat([df_all, df])
    return df_all

df_all = read_all_reviews_in_area(SF_review_files)
df_all.head()            

,review_name,review_rating,review_date,review_text,biz_url,biz_rating,biz_name
0,Emily F.,5.0,2020-03-20,"We are so grateful for Dan, Natasha, Aaron, an...",/biz/dan-perata-dog-training-san-francisco?osq...,4.5,Dan Perata Dog Training
1,Kelly W.,5.0,2020-05-28,"Dear DPT,\n\nJust wanted to reach out and send...",/biz/dan-perata-dog-training-san-francisco?osq...,4.5,Dan Perata Dog Training
2,Vicki L.,5.0,2020-03-31,My husband and I are very happy with the train...,/biz/dan-perata-dog-training-san-francisco?osq...,4.5,Dan Perata Dog Training
3,Chris D.,5.0,2020-03-19,Dan and Natasha make you and your pet feel lik...,/biz/dan-perata-dog-training-san-francisco?osq...,4.5,Dan Perata Dog Training
4,AdventureOrNothing K.,5.0,2019-09-17,This place and the trainers and staff are so e...,/biz/dan-perata-dog-training-san-francisco?osq...,4.5,Dan Perata Dog Training


In [22]:
df_all.shape

(106243, 7)

In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   review_name    5 non-null      object 
 1   review_rating  5 non-null      float64
 2   review_date    5 non-null      object 
 3   review_text    5 non-null      object 
 4   biz_url        5 non-null      object 
 5   biz_rating     5 non-null      float64
 6   biz_name       5 non-null      object 
dtypes: float64(2), object(5)
memory usage: 408.0+ bytes
None


In [24]:
# categorical
object_cols = [col for col in df.columns if df[col].dtype == "object"]

# find unique biz_name
df["biz_name"].unique()

array(['Dan Perata Dog Training'], dtype=object)

In [25]:
# check number of duplicatesa
print(len(df[df.duplicated()]))

0


In [26]:
# missing val
print(df.isna().any())

review_name      False
review_rating    False
review_date      False
review_text      False
biz_url          False
biz_rating       False
biz_name         False
dtype: bool


In [28]:
df_all.loc[df_all.review_text.str.len().min()].review_text

39    My big head Bruno loves going to Pet Camp!! As...
39    Wag Hotel San Francisco supports Tubb fire vic...
39    Finally getting around to writing my review! C...
39    Wag Hotel San Francisco supports Tubb fire vic...
39    Horrible experience with one of their vets. I ...
39    Amazing staff who has saved our fur baby on ma...
39    Nina is great! My pitbull is super energetic a...
39    Wag Hotel San Francisco supports Tubb fire vic...
39    I've been going to this place for years, and I...
39    Wonderful, small intimate clinic for cats. My ...
39    As I begin to travel more, I have grown concer...
39    Amazing work! We have a very antisocial long h...
39    We loved Erika but overall the experience was ...
39    Great place to get a pet bird if you are in th...
39    My big head Bruno loves going to Pet Camp!! As...
39    My big head Bruno loves going to Pet Camp!! As...
39    My big head Bruno loves going to Pet Camp!! As...
39    We LOVE the Diggity Dog team!!  Donna and 

In [29]:
df = df_all

In [30]:
for text in df.review_text.iloc[:50]:
    print(text)
    print("---")

We are so grateful for Dan, Natasha, Aaron, and the entire Dan Perata staff -- they are amazing and I recommend them to EVERYONE.

Our puppy just went through Dan Camp -- and she is like a new dog. When we brought her in, we were struggling with her training. She had no on-leash skills, she was barking, having accidents...the list goes on. In just four weeks, we can take her on walks easily, she knows all of her commands, and easily sleeps through the night in her crate without any accidents. 

If you want results, all you need to do is come to Dan Perata and follow their recommendations. They invest in you and your dog, and if you do the work, they will make sure you have a great puppy coming out of camp. 

Lastly, they are professional and easy to work with on the operational side: scheduling additional boarding, grooming, checking-in...it's all simple and effective.

I am so impressed by them. Thank you Dan Perata Training!
---
Dear DPT,

Just wanted to reach out and send a special 

In [31]:
ori_colnames = df.columns
print(ori_colnames)

Index(['review_name', 'review_rating', 'review_date', 'review_text', 'biz_url',
       'biz_rating', 'biz_name'],
      dtype='object')


### Remove reviews only mention grooming or product sales, but not boarding/hotel/kennel-type services

In [33]:
def get_number_of_reviews(df):
    import re
    print("# of reviews containing word board: ",
          df.review_text.str.contains(re.compile(r'board(?i)')).sum())
    print("# of reviews containing word -cation: ",
          df.review_text.str.contains(re.compile(r'cation(?i)')).sum())
    print("# of reviews containing word hotel: ",
          df.review_text.str.contains(re.compile(r'hotel(?i)')).sum())
    print("# of reviews containing word kennel: ",
          df.review_text.str.contains(re.compile(r'kennel(?i)')).sum())
    print("# of reviews containing word travel: ",
          df.review_text.str.contains(re.compile(r'travel(?i)')).sum())
    print("# of reviews containing word sitter: ",
          df.review_text.str.contains(re.compile(r'sitter(?i)')).sum())
    print("# of reviews containing word sitting: ",
          df.review_text.str.contains(re.compile(r'sitting(?i)')).sum())
    print("# of reviews containing phrase day care: ",
          df.review_text.str.contains(re.compile(r'day care(?i)')).sum())
    print("# of reviews containing word groom: ",
          df.review_text.str.contains(re.compile(r'groom(?i)')).sum())

In [34]:
get_number_of_reviews(df)

# of reviews containing word board:  9490
# of reviews containing word -cation:  12779
# of reviews containing word hotel:  824
# of reviews containing word kennel:  1455
# of reviews containing word travel:  4212
# of reviews containing word sitter:  5543
# of reviews containing word sitting:  4640
# of reviews containing phrase day care:  1314
# of reviews containing word groom:  7573


In [36]:
def remove_irrelevant_reviews(df):
    import re
    long_con = (df.review_text.str.contains(re.compile(r'kennel(?i)')) & ~(df.review_text.str.contains(re.compile(r'kennel cough(?i)'))))

    condition = ((df.review_text.str.contains(re.compile(r'board(?i)')))
                 | (df.review_text.str.contains(re.compile(r'cation(?i)')))
                 | (df.review_text.str.contains(re.compile(r'hotel(?i)')))
                 | (df.review_text.str.contains(re.compile(r'travel(?i)')))
                 | (df.review_text.str.contains(re.compile(r'sitter(?i)')))
                 | (df.review_text.str.contains(re.compile(r'sitting(?i)')))
                 | (df.review_text.str.contains(re.compile(r'day care(?i)')))
                 | long_con)
    df = df[condition]
    return df

df_new = remove_irrelevant_reviews(df)
df_new.shape

(30696, 7)

## Sanity check to make sure no reviews are explicity on groom only

In [37]:
import re
long_con = (df.review_text.str.contains(re.compile(r'kennel(?i)')) & ~(df.review_text.str.contains(re.compile(r'kennel cough(?i)'))))

condition = ((df.review_text.str.contains(re.compile(r'board(?i)')))
             | (df.review_text.str.contains(re.compile(r'cation(?i)')))
             | (df.review_text.str.contains(re.compile(r'hotel(?i)')))
             | (df.review_text.str.contains(re.compile(r'travel(?i)')))
             | (df.review_text.str.contains(re.compile(r'sitter(?i)')))
             | (df.review_text.str.contains(re.compile(r'sitting(?i)')))
             | (df.review_text.str.contains(re.compile(r'day care(?i)')))
             | long_con)

text = df_new[(df_new.review_text.str.contains(re.compile(r'groom(?i)'))) & 
               ~(condition) &
               ~(long_con)]['review_text']

assert(len(text) == 0)

KeyboardInterrupt: 

In [ ]:
df_new.head()

In [ ]:
df_new[df_new.review_rating == 1.0].biz_name.value_counts()

# NLP cleaning for model
* Remove all irrelevant characters e.g., non alphanumeric characters
* Remove words that are not relevant, such as urls or links
* Remove white spaces
* Remove accents
* stop words beyond standard NLTK stop words needed to be removed to make the model more robust
     - These words included days of the week and their abbreviations, month names, and the word “Yelp"
* [depends] Convert characters to lowercase, in order to treat words such as “hello”, “Hello”, and “HELLO” the same.. But Bill != bill
* Consider combining misspelled or alternately spelled words to a single representation (e.g. “cool”/”kewl”/”cooool”)
* Consider lemmatization (reduce words such as “am”, “are”, and “is” to a common form such as “be”)
    - It is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. For example, “include”, “includes,” and “included” would all be represented as “include”.
* create word cloud to examine frequent but useless words
* Negations needed to be dealt with
  - creating a dictionary of negations to handle negated words
* expand contractions, remove links, hashtags, capitalization, and punctuation

Finally, Tokenize text by separating it into individual words.

In [ ]:
df_new['review_text'].str.contains('\n').sum()

In [ ]:
df_new['review_text'][df_new['review_text'].str.contains('\n')]

In [ ]:
df_new['review_text'][df_new['review_text'].str.contains('\r')]

In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "") # followed by one or more non-whitespaces, for the domain name
    # df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"&", "and")
    df[text_field] = df[text_field].str.replace(r"#", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at ")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\''\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r'\d+', ' ')     
    # but want to keep e.g., 18 years old, how would this info be captured?
    
    df[text_field] = df[text_field].str.replace(r"\'", " ")
    df[text_field] = df[text_field].str.replace(r"\"", " ")
    df[text_field] = df[text_field].str.replace(r"\n", " ")
    df[text_field] = df[text_field].str.replace(r"\r", " ")
    df[text_field] = df[text_field].str.replace(r"bc", "because")
    df[text_field] = df[text_field].str.replace(r"b c", "because")
    df[text_field] = df[text_field].str.replace(r"b/c", "because")
    df[text_field] = df[text_field].str.replace(r"dr\.", "dr")
    df[text_field] = df[text_field].str.replace(r"d\.r\.", "dr")
    df[text_field] = df[text_field].str.replace(r"tbh", "to be honest")
    df[text_field] = df[text_field].str.lower()    
    return df



In [ ]:
from contractions import CONTRACTION_MAP
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def clean_text(text):
    import re
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    # text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"hr", "hour", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    # text = re.sub('\W', ' ', text)     # any non-alphanumeric character. Equivalent to [^a-zA-Z0-9_]
     #text = re.sub('\s+', ' ', text)    # all whitespaces
    text = text.strip(' ')
    return text

def clean_text_2(text):
    '''Eemove text in square brackets, remove punctuation, remove words containing numbers.'''
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)
    
def clean_text_3(text):
    """ Reomve Yelp""" 
    text = re.sub(r'yelp(?i)', ' ', text)
    text = re.sub(r"(mon|tues|wednes|thurs|fri|satur|sun)day", " ", text)    
    return text
    
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text



In [ ]:
df_new = standardize_text(df_new, 'review_text')
df_new['review_text'] = df_new['review_text'].map(lambda x : expand_contractions(x))
df_new['review_text'] = df_new['review_text'].map(lambda x : clean_text(x))
df_new['review_text'] = df_new['review_text'].apply(lambda x: clean_text_2(x))
df_new['review_text'] = df_new['review_text'].apply(lambda x: clean_text_3(x))
df_new['review_text'] = df_new['review_text'].apply(lambda x: remove_accented_chars(x))

for text in df_new.review_text.iloc[:50]:
    print(text)
    print("---")


In [ ]:
df_new['review_text'].iloc[0]

## Lemmatization

In [ ]:
# use spaCy for lemmatization
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['ner', 'parser'])   # don't need, disable to speed up

def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [ ]:
df_new['review_text_lem'] = df_new['review_text'].apply(lambda x: lemmatizer(x))
df_new['review_text_lem_cleaned'] = df_new['review_text_lem'].str.replace('-PRON-', '')

In [ ]:
# get top 10 most frequent words
sentences = [row.split() for row in df_new.review_text_lem_cleaned]
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
        
sorted(word_freq, key=word_freq.get, reverse=True)[:10]


In [ ]:
# get top 10 most frequent words for lowest reviews
sentences = [row.split() for row in df_new.review_text_lem_cleaned[df_new.review_rating < 3]]
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
        
sorted(word_freq, key=word_freq.get, reverse=True)[:10]


In [ ]:
# get top 10 most frequent words for best reviews
sentences = [row.split() for row in df_new.review_text_lem_cleaned[df_new.review_rating > 3]]
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
        
sorted(word_freq, key=word_freq.get, reverse=True)[:10]


## Tokenize

In [ ]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

df_new['review_text_lem_cleaned_tokenized'] = df_new['review_text_lem_cleaned'].apply(lambda x: tokenize(x.lower()))
df_new['review_text_lem_cleaned_tokenized'].head()


In [ ]:
for text in df_new.review_text_lem_cleaned_tokenized.iloc[:50]:
    print(text)
    print("---")

In [ ]:
def remove_stopwords(tokenized_list):
    """
    Done above when cleaning, but just to make sure
    """
    text = [word for word in tokenized_list if word not in STOPWORDS]
    return text

df_new['review_text_lem_cleaned_tokenized_nostop'] = df_new['review_text_lem_cleaned_tokenized'].apply(lambda x: remove_stopwords(x))
df_new['review_text_lem_cleaned_tokenized_nostop'].equals(df_new['review_text_lem_cleaned_tokenized'])

In [ ]:
for text in df_new.review_text_lem_cleaned_tokenized_nostop.iloc[:50]:
    print(text)
    print("---")

## WordCloud

In [ ]:
# Word cloud, more stopwords?
from wordcloud import WordCloud
text = df_new['review_text_lem_cleaned_tokenized_nostop'].values
wordcloud = WordCloud(
    width=3000,
    height=2000,
    background_color='black', max_words=200, 
    stopwords=STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize=(40, 30),
    facecolor='k',
    edgecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Word cloud for bad reviews 
from wordcloud import WordCloud
text = df_new[df_new.review_rating < 3]['review_text_lem_cleaned_tokenized_nostop'].values
wordcloud = WordCloud(
    width=3000,
    height=2000,
    background_color='black', max_words=200, 
    stopwords=STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize=(40, 30),
    facecolor='k',
    edgecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Word cloud for good reviews
from wordcloud import WordCloud
text = df_new[df_new.review_rating > 3]['review_text_lem_cleaned_tokenized_nostop'].values
wordcloud = WordCloud(
    width=3000,
    height=2000,
    background_color='black', max_words=200, 
    stopwords=STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize=(40, 30),
    facecolor='k',
    edgecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## top words in rating=5 and rating=1s

In [ ]:
df_05 = df_new.loc[df_new['review_rating'] == 5]
df_01 = df_new.loc[df_new['review_rating'] == 1]

In [ ]:
top_wrd5 = df_05['review_text_lem_cleaned'].str.split(
    expand=True).stack().value_counts()
top_wrd5 = pd.DataFrame(top_wrd5)
top_wrd5['Index'] = top_wrd5.index
top_wrd5.columns = ["y", "x"]
top_wrd5 = top_wrd5.head(10)

top_wrd1 = df_01['review_text_lem_cleaned'].str.split(
    expand=True).stack().value_counts()
top_wrd1 = pd.DataFrame(top_wrd1)
top_wrd1['Index'] = top_wrd1.index
top_wrd1.columns = ["y", "x"]
top_wrd1 = top_wrd1.head(10)
#[word for word in data2['tokens'] if word not in stop_words]

p5 = top_wrd5.plot.bar(x="x", y="y")
p1 = top_wrd1.plot.bar(x="x", y="y")

# Topic modeling
Identify recurrent themes or topics.

![](https://github.com/xiey1/OncoMatch/raw/master/images/Word_embedding.png)
The embedding performance is plotted as the distribution of cosine similarity score rankings between each abstract and its corresponding title (better performance corresponds to higher ranking and percentile closer to 0).

# Sentence similarities -- Word embedding models

Skipping basic models becuase Bag-of-words models are surprisingly effective, but have several weaknesses.

    1. First, they lose all information about word order: “John likes Mary” and “Mary likes John” correspond to identical vectors. There is a solution: bag of n-grams models consider word phrases of length n to represent documents as fixed-length vectors to capture local word order but suffer from data sparsity and high dimensionality.

    2. Second, the model does not attempt to learn the meaning of the underlying words, and as a consequence, the distance between vectors doesn’t always reflect the difference in meaning. The Word2Vec model addresses this second problem.


* Static vs Dynamic
    * Static:
        * word has same vector regardless of context 
        * Look at word from left to right (unidirectional) 
        * e.g., GloVe (Stanford, 2014), FastText (FB, 2015)
    * Dynamic: 
        * Understand a word as part of the sentence
        * Take into account context
        * Different vectors for the same word
        * Look at word from left to right and also opposite direction 
            * So word doesn’t depend on the sequence, but get the context in a sentence
        * e.g., Siri, google translation 
        * e.g., ELMo (UW, 2018), BERT (Google 2018), OpenAI GPT, OpenAI GPT-2
        
* Two architectures: 
    * Common bag of word (CBOW)
        * Use surrounding words (context) to predict target word
        * Cat & dog would be similar —> not so accurate
    * Skip gram
        * Use target word to predict context
        * Weights close words more heavily
        * Cat and dog would be different —> more fined-tuned vectors, but takes longer time
    * CBOW does better on small data sets, skip-gram better on large ones  
    

If I want to calc a vector for the entire paragraph/user input, we could:
    1. average the vectors for each word in the review
    2. use paragraphh vector (`Doc2Vec`)
    ,...... https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
    
Doc2Vec:
- Unlike Word2Vec, a Doc2Vec model provides a vectorised representation of a group of words taken collectively as a single unit. It is not a simple average of the word vectors of the words in the sentence.
    
To try for this project: Word2Vec --> Doc2Vec --> BERT --> GenSen

## w2v w/ skip-gram :

The basic idea of Word2vec is that instead of representing words as one-hot encoding (countvectorizer / tfidfvectorizer) in high dimensional space, we represent words in dense low dimensional space in a way that similar words get similar word vectors, so they are mapped to nearby points.

Word2vec is not deep neural network, it turns text into a numeric form that deep neural network can process as input.

When we use Tensorflow / Keras or Pytorch to do this, they have a special layer for this process called “Embedding layer”. So, we are not going to do math by ourselves, we only need to pass one-hot encoded vectors, the “Embedding layer” does all the dirty works.

Word2Vec is a technique to find continuous embeddings for words. It learns from reading massive amounts of text and memorizing which words tend to appear in similar contexts. After being trained on enough data, it generates a 300-dimension vector for each word in a vocabulary, with words of similar meaning being closer to each other.


Trained on 100 billion words from google news data, 300 dimensions
![](https://miro.medium.com/max/1400/1*9XxSnprkyh2I7Jts4lpxAg.png)
Source: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
https://towardsdatascience.com/understanding-word2vec-embedding-in-practice-3e9b8985953

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    """
    calc a vector for the entire paragraph/user input by averaging 
    vectors for each word in the entry/paragraph.
    
    """
    if len(tokens_list) < 1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(
            k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(
            k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged


def get_word2vec_embeddings(vectors, df_list_tokens, generate_missing=False, k=300):
    embeddings = df_list_tokens.apply(lambda x: get_average_word2vec(x, vectors,
                                                             generate_missing=generate_missing,
                                                                    k=k))
    return list(embeddings)

### Train on own data

Training your own word vectors may be the best approach for a given NLP problem.

But it can take a long time, a fast computer with a lot of RAM and disk space, and perhaps some expertise in finessing the input data and training algorithm.

- min_count: Minimum number of occurrences of a word in the corpus to be included in the model. The higher the number, the less words we have in our corpus.
- window: The maximum distance between the current and predicted word within a sentence.
- size: The dimensionality of the feature vectors.
- iter: epoch

In [ ]:
# save all the cleaned reviews, tokens joined by whitespaces in a filename "scraped_data_nodupl_biz/reviews_corpus.cor"
df_new.to_csv('scraped_data_nodupl_biz/SF_reviews_corpus.cor', ignore_index=True, 
              index=False)

In [ ]:
"""
Helper func to handle large corpus line by line

If we wanted to do any custom preprocessing, 
e.g. decode a non-standard encoding, lowercase, remove numbers, extract named entities...
All of this can be done inside the MyCorpus iterator and word2vec doesn’t need to know. 
All that is required is that the input yields one sentence (list of utf8 words) after another.

"""

from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self, fname):
        corpus_path = datapath(fname)
        for line in open(corpus_path):
            # assume there's one document/review per line, where the pre-processed tokens separated by whitespace
            yield utils.simple_preprocess(line)            

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Remove-reviews-only-mention-grooming-or-product-sales,-but-not-boarding/hotel/kennel-type-services" data-toc-modified-id="Remove-reviews-only-mention-grooming-or-product-sales,-but-not-boarding/hotel/kennel-type-services-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Remove reviews only mention grooming or product sales, but not boarding/hotel/kennel-type services</a></span></li></ul></li><li><span><a href="#Sanity-check-to-make-sure-no-reviews-are-explicity-on-groom-only" data-toc-modified-id="Sanity-check-to-make-sure-no-reviews-are-explicity-on-groom-only-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sanity check to make sure no reviews are explicity on groom only</a></span></li></ul></li><li><span><a href="#NLP-cleaning-for-model" data-toc-modified-id="NLP-cleaning-for-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>NLP cleaning for model</a></span><ul class="toc-item"><li><span><a href="#Lemmatization" data-toc-modified-id="Lemmatization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Lemmatization</a></span></li><li><span><a href="#Tokenize" data-toc-modified-id="Tokenize-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Tokenize</a></span></li><li><span><a href="#WordCloud" data-toc-modified-id="WordCloud-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>WordCloud</a></span></li><li><span><a href="#top-words-in-rating=5-and-rating=1s" data-toc-modified-id="top-words-in-rating=5-and-rating=1s-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>top words in rating=5 and rating=1s</a></span></li></ul></li><li><span><a href="#Topic-modeling" data-toc-modified-id="Topic-modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Topic modeling</a></span></li><li><span><a href="#Sentence-similarities----Word-embedding-models" data-toc-modified-id="Sentence-similarities----Word-embedding-models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sentence similarities -- Word embedding models</a></span><ul class="toc-item"><li><span><a href="#w2v-w/-skip-gram-:" data-toc-modified-id="w2v-w/-skip-gram-:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>w2v w/ skip-gram :</a></span><ul class="toc-item"><li><span><a href="#Train-on-own-data" data-toc-modified-id="Train-on-own-data-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Train on own data</a></span></li></ul></li></ul></li><li><span><a href="#Analysis/Modeling" data-toc-modified-id="Analysis/Modeling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analysis/Modeling</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Conclusions-and-Next-Steps" data-toc-modified-id="Conclusions-and-Next-Steps-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Conclusions and Next Steps</a></span></li></ul></div>